In [1]:
import os
import io
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, Image as IPImage
from datetime import datetime, timedelta
from scripts.utils import load_scenario_data
from src.environment import AircraftDisruptionOptimizer
from scripts.visualizations import StatePlotter
from scripts.utils import parse_time_with_day_offset
from src.config import *

# Set the same timestep as in the DQN environment
TIMESTEP_HOURS = 1  # Adjust if needed

def run_exact_solution_and_plot(scenario_folder, plot_title="Exact Solution"):
    data_dict = load_scenario_data(scenario_folder)
    aircraft_dict = data_dict['aircraft']
    flights_dict = data_dict['flights']
    rotations_dict = data_dict['rotations']
    alt_aircraft_dict = data_dict['alt_aircraft']
    config_dict = data_dict['config']

    # Parse recovery period times
    recovery_period = config_dict['RecoveryPeriod']
    start_datetime = datetime.strptime(
        f"{recovery_period['StartDate']} {recovery_period['StartTime']}", '%d/%m/%y %H:%M'
    )
    end_datetime = datetime.strptime(
        f"{recovery_period['EndDate']} {recovery_period['EndTime']}", '%d/%m/%y %H:%M'
    )

    optimizer = AircraftDisruptionOptimizer(
        aircraft_dict=aircraft_dict,
        flights_dict=flights_dict,
        rotations_dict=rotations_dict,
        alt_aircraft_dict=alt_aircraft_dict,
        config_dict=config_dict
    )

    # Initialize uncertain breakdowns
    uncertain_breakdowns = {}
    for ac_id, ac_info in alt_aircraft_dict.items():
        if 'UnavailabilityProbability' in ac_info:
            uncertain_breakdowns[ac_id] = {
                'probability': ac_info['UnavailabilityProbability'],
                'duration': ac_info['UnavailabilityDuration'],
                'is_broken': False,
                'breakdown_start': None,
                'breakdown_end': None
            }

    state_plotter = StatePlotter(
        aircraft_dict=aircraft_dict,
        flights_dict=flights_dict,
        rotations_dict=rotations_dict,
        alt_aircraft_dict=alt_aircraft_dict,
        start_datetime=start_datetime,
        end_datetime=end_datetime,
        uncertain_breakdowns=uncertain_breakdowns,
        plot_title=plot_title
    )

    # Solve the problem once and store the solution
    solution = optimizer.solve()

    # Extract solution details and action history
    cancellations = set(solution['cancellations'])
    delays = solution['delays']
    assignments = solution['assignments']  # flight_id -> new aircraft
    action_history = optimizer.action_history

    # Create working copies of the dictionaries so we don't modify originals
    flights_dict_working = {k: v.copy() for k, v in flights_dict.items()}
    rotations_dict_working = {k: v.copy() for k, v in rotations_dict.items()}
    alt_aircraft_dict_working = {k: v.copy() for k, v in alt_aircraft_dict.items()}

    # Lists to track applied actions at each timestep
    plots = []
    action_logs = []  # Store text descriptions of actions
    current_breakdowns = set()  # Track current breakdowns

    def capture_plot_and_actions(title_suffix, current_datetime, actions_this_step, total_reward=0):
        # Get current swapped flights
        swapped_flights = []
        for f_id, f_info in flights_dict_working.items():
            if 'NewAircraft' in f_info:
                swapped_flights.append((f_id, f_info['NewAircraft']))
        
        # Get current delayed and cancelled flights
        environment_delayed_flights = {f_id for f_id, f_info in flights_dict_working.items() 
                                    if 'Delay' in f_info and f_info['Delay'] > 0}
        cancelled_flights = {f_id for f_id, f_info in flights_dict_working.items() 
                           if 'Cancelled' in f_info and f_info['Cancelled']}

        # Update the StatePlotter's dictionaries with the working copies
        state_plotter.alt_aircraft_dict = alt_aircraft_dict_working
        state_plotter.flights_dict = flights_dict_working
        state_plotter.rotations_dict = rotations_dict_working

        # Plot the current state
        fig = state_plotter.plot_state(
            flights_dict_working,
            swapped_flights=swapped_flights,
            environment_delayed_flights=environment_delayed_flights,
            cancelled_flights=cancelled_flights,
            current_datetime=current_datetime,
            title_appendix=f"{title_suffix}\n{actions_this_step}",
            show_plot=False,
            reward_and_action=(0, "No action", total_reward)  # Added reward display
        )
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        img = IPImage(data=buf.read(), format='png', embed=True)
        plots.append(img)
        action_logs.append(actions_this_step)
        plt.close(fig)

    # Initial state plot
    capture_plot_and_actions("Initial State", start_datetime, "No actions yet")

    # Generate timesteps matching action history length
    time_steps = [start_datetime + timedelta(hours=i*TIMESTEP_HOURS) for i in range(len(action_history)+1)]

    total_reward = 0
    # Apply solution progressively based on action history
    for idx, t_step in enumerate(time_steps[1:], 1):  # Skip first timestep since it's initial state
        actions_this_step = []
        
        # Update uncertain breakdowns
        for ac_id, breakdown_info in uncertain_breakdowns.items():
            if not breakdown_info['is_broken']:
                # Check for new breakdown
                if np.random.random() < breakdown_info['probability']:
                    breakdown_info['is_broken'] = True
                    breakdown_info['breakdown_start'] = t_step
                    breakdown_info['breakdown_end'] = t_step + timedelta(hours=breakdown_info['duration'])
                    current_breakdowns.add(ac_id)
                    actions_this_step.append(f"Aircraft {ac_id} broke down!")
            else:
                # Check if breakdown is over
                if t_step >= breakdown_info['breakdown_end']:
                    breakdown_info['is_broken'] = False
                    current_breakdowns.remove(ac_id)
                    actions_this_step.append(f"Aircraft {ac_id} repaired!")

        # Update alt_aircraft_dict with current breakdowns
        for ac_id in alt_aircraft_dict_working:
            if ac_id in current_breakdowns:
                alt_aircraft_dict_working[ac_id]['IsAvailable'] = False
            else:
                alt_aircraft_dict_working[ac_id]['IsAvailable'] = True
        
        # Get action from history for this timestep
        action = action_history[idx-1]
        flight_id = action['flight']
        aircraft_id = action['aircraft']
        reward = action['reward']
        conflicts = action['conflicts']
        
        total_reward += reward
        
        actions_this_step.append(f"Step {idx}:")
        actions_this_step.append(f"Flight: {flight_id}, Aircraft: {aircraft_id}")
        actions_this_step.append(f"Reward: {reward:.1f}, Conflicts: {conflicts}")
        
        if flight_id > 0:  # Non-zero flight ID means an actual action was taken
            # Apply the action
            if flight_id in flights_dict_working:
                if aircraft_id > 0:  # Reassignment
                    old_ac = rotations_dict[flight_id]['Aircraft']
                    new_ac = f'B737#{aircraft_id}'
                    flights_dict_working[flight_id]['NewAircraft'] = new_ac
                    rotations_dict_working[flight_id]['Aircraft'] = new_ac
                    actions_this_step.append(f"Reassigned flight {flight_id} from {old_ac} to {new_ac}")

        action_text = "\n".join(actions_this_step)
        capture_plot_and_actions(f"Step {idx}", t_step, action_text, total_reward)

    # Create widgets for navigation
    output = widgets.Output()

    def update_display(index):
        with output:
            clear_output(wait=True)
            display(plots[index])
            print("\nActions at this step:")
            print(action_logs[index])

    slider = widgets.IntSlider(
        value=0,
        min=0,
        max=len(plots)-1,
        step=1,
        description='Step:'
    )

    def on_slider_change(change):
        if change['name'] == 'value':
            update_display(change['new'])

    slider.observe(on_slider_change, names='value')

    prev_button = widgets.Button(description='⬅️', layout=widgets.Layout(width='40px'))
    next_button = widgets.Button(description='➡️', layout=widgets.Layout(width='40px'))

    def on_prev_button_clicked(b):
        slider.value = max(0, slider.value - 1)

    def on_next_button_clicked(b):
        slider.value = min(len(plots)-1, slider.value + 1)

    prev_button.on_click(on_prev_button_clicked)
    next_button.on_click(on_next_button_clicked)

    navigation = widgets.HBox([prev_button, next_button, slider])
    update_display(0)
    display(navigation, output)

    # Print final solution statistics
    print("\nOptimization Results:")
    print(f"Objective value: {solution['objective_value']:.2f}")
    print(f"\nSolution statistics:")
    print(f"  Runtime: {solution['statistics']['runtime']:.2f} seconds")
    print(f"  Status: {solution['statistics']['status']}")
    print(f"\nSolution summary:")
    print(f"  Cancelled flights: {len(solution['cancellations'])}")
    print(f"  Total delay minutes: {solution['total_delay_minutes']}")
    print(f"  Number of reassignments: {len(solution['assignments'])}")
    print(f"  Total reward: {total_reward:.2f}")

# Example usage in a Jupyter notebook:
scenario_folder = "../data/Training/6ac-700-diverse/mixed_high_Scenario_001"
run_exact_solution_and_plot(scenario_folder)


  Action 0 (flight=0, aircraft=0): reward=-5120.0
    -> New best action (reward=-5120.0)
  Action 7 (flight=1, aircraft=0): reward=-10114.0
  Action 8 (flight=1, aircraft=1): reward=-5120.0
  Action 9 (flight=1, aircraft=2): reward=-7614.0
  Action 10 (flight=1, aircraft=3): reward=-7614.0
  Action 11 (flight=1, aircraft=4): reward=-7614.0
  Action 12 (flight=1, aircraft=5): reward=-7614.0
  Action 13 (flight=1, aircraft=6): reward=-5114.0
    -> New best action (reward=-5114.0)
  Action 14 (flight=2, aircraft=0): reward=-10085.0
  Action 15 (flight=2, aircraft=1): reward=-5120.0
  Action 16 (flight=2, aircraft=2): reward=-7585.0
  Action 17 (flight=2, aircraft=3): reward=-7585.0
  Action 18 (flight=2, aircraft=4): reward=-7585.0
  Action 19 (flight=2, aircraft=5): reward=-7585.0
  Action 20 (flight=2, aircraft=6): reward=-6212.0
  Action 21 (flight=3, aircraft=0): reward=-10056.2
  Action 22 (flight=3, aircraft=1): reward=-5120.0
  Action 23 (flight=3, aircraft=2): reward=-7252.7
  A

Output()


Optimization Results:
Objective value: 1190.20

Solution statistics:
  Runtime: 0.34 seconds
  Status: Complete

Solution summary:
  Cancelled flights: 0
  Total delay minutes: 0
  Number of reassignments: 0
  Total reward: -1190.20
